# Chatbot with Your Doc

# Medical Q&A Assistant

In [1]:
!pip install --upgrade langchain langchain-huggingface langchain-community faiss-cpu sentence-transformers transformers


In [2]:
!pip install pypdf

In [3]:
!pip uninstall -y transformers
!pip install transformers

Found existing installation: transformers 4.53.0
Uninstalling transformers-4.53.0:
  Successfully uninstalled transformers-4.53.0
  Using cached transformers-4.53.0-py3-none-any.whl.metadata (39 kB)
Using cached transformers-4.53.0-py3-none-any.whl (10.8 MB)


In [4]:
!pip install --upgrade streamlit langchain langchain-huggingface langchain-community faiss-cpu sentence-transformers transformers pypdf


In [5]:
# Imports
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_core.documents import Document # Import Document class
from transformers import AutoTokenizer
# from langchain_community.embeddings.johnsnowlabs import JohnSnowLabsEmbeddings
# 1. Load your medical text corpus
# Place your medical documents into a file named data.txt, one file per line.
loader = PyPDFLoader("/content/DPTX_2012_1_11160_0_271561_0_118026.pdf")
docs = loader.load()


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

def truncate_chunk(text, max_tokens=400):
    tok = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_tokens)
    return tokenizer.decode(tok["input_ids"][0])



# 2. Split into chunks for retrieval
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
chunks = splitter.split_documents(docs)

# Truncate text and recreate Document objects
truncated_documents = []
for doc in chunks:
  truncated_text = truncate_chunk(doc.page_content)
  truncated_documents.append(Document(page_content=truncated_text, metadata=doc.metadata))

# 3. Convert chunks to embeddings using a Hugging Face Sentence Transformer
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embedder = JohnSnowLabsEmbeddings("en.embed_sentence.biobert.clinical_base_cased")


# 4. Store embeddings in FAISS vector database
db = FAISS.from_documents(truncated_documents, embeddings)

# 5. Use Hugging Face's FLAN-T5-large model locally for text2text generative chain
llm = HuggingFacePipeline.from_model_id(
  model_id="google/flan-t5-large",
  task="text2text-generation",
  pipeline_kwargs={"max_new_tokens": 512, "temperature": 0.1}
)

# 6. Create RetrievalQA chain with source retrieval
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    return_source_documents=True
)
# 7. Ask clinical questions
def ask_medical_question(query: str):
    result = qa.invoke({"query": query})
    print("🧠 Answer:")
    print(result["result"], "\n")
    print("📚 Sources:")
    for doc in result["source_documents"]:
        snippet = doc.page_content[:200].replace("\n", " ")
        print("—", snippet, "...")
    print()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [7]:
# Example usage
ask_medical_question("What are the common symptoms of tuberculosis?")
ask_medical_question("How is type 2 diabetes diagnosed?")

Token indices sequence length is longer than the specified maximum sequence length for this model (1662 > 512). Running this sequence through the model will result in indexing errors
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧠 Answer:
rhino-cerebral mucormycosis and community-acquired pneumonia are among the most common infections the diabetic patients suffer from 

📚 Sources:
— 29 rhino-cerebral mucormycosis and community-acquired pneumonia are among the most common infections the diabetic patients suffer from (Gu, 1998). Recommendations for managing the complications of dia ...
— 17 causes severe dehydration. At the same time, without enough insulin to allow sugar absorption, the body's cells act as if they are starving. Without insulin, patients with type I diabetes develop s ...
— 10 An oral glucose tolerance test (OGTT) measures blood glucose after a person fasts at least 8 hours and 2 hours after the person drinks a glucose -containing beverage. This test can be used to diagn ...
— 7 1.1 Definition Diabetes mellitus, or simply diabetes, is a group of diseases characterized by high blood glucose levels that result from defects in the body's ab ility to produce and/or use insulin. ...



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧠 Answer:
FPG test is the preferred test for diagnosing diabetes because of its convenience and low cost. However, it may miss some diabetes or prediabetes that can be found with the OGTT. The FPG test is most reliable when done in the morning. Research has shown that the OGTT is more sensitive than the FPG test for diagnosing prediabetes. Test results indicating that a person has diabetes should be conf irmed with a second test on a different day (Twillman, 2002). The current WHO diagnostic criteria for diabetes should be maintained – fasting plasma glucose  7.0mmol/l (126mg/dl) or 2 –h plasma glucose  11.1mmol/l (200mg/dl) (Report of a WHO Consultation, 1999). 

📚 Sources:
— 8 diagnose them with diabetes mellitus -"honeyed" diabetes. This method of monitoring blood sugars went largely unchanged until the 20th century. Before the discovery of the insulin little could be do ...
— 10 An oral glucose tolerance test (OGTT) measures blood glucose after a person fasts at least 8 hours and 2

It looks like the ngrok authentication failed. Please follow these steps to get a valid authtoken:

1. Go to [ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken) and sign up for a free account.
2. Copy your authtoken from the dashboard.
3. In Colab, click on the "🔑" icon in the left sidebar to open the Secrets manager.
4. Click on "New secret".
5. For the "Name" of the secret, enter `NGROK_TOKEN`.
6. For the "Value" of the secret, paste your authtoken.
7. Close the Secrets manager.

Once you have added the secret, try running the cell again.

In [ ]:
%%writefile app.py
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import HuggingFacePipeline

from langchain.chains import RetrievalQA
import os

# 🔧 UI
st.set_page_config(page_title="🩺 Medical Q&A Assistant")
st.title("🩺 Medical Q&A Assistant")
st.subheader("Upload a medical PDF")

# 🗂️ File Upload
uploaded_file = st.file_uploader("Upload PDF", type=['pdf'])

if uploaded_file is not None:
    # ✅ Save uploaded file locally
    with open(uploaded_file.name, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.success("✅ File Uploaded Successfully")

    # 📄 Load PDF
    loader = PyPDFLoader(uploaded_file.name)
    documents = loader.load()

    # 🔗 Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    # 🧠 Create embeddings  =====>  sentence-transformers/all-mpnet-base-v2
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # 🗃️ Create FAISS vector DB
    db = FAISS.from_documents(texts, embeddings)

    # 🔍 Setup retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    
    llm = HuggingFacePipeline.from_model_id(
      model_id="google/flan-t5-large",
      task="text2text-generation",
      pipeline_kwargs={"max_new_tokens": 512, "temperature": 0.1}
    )


    # 🔗 Create QA Chain
    qa = RetrievalQA.from_chain_type(
        llm=llm, retriever=retriever, chain_type="stuff"
    )

    # 💬 User input
    query = st.text_input("Ask a question about the PDF:")

    if query:
        with st.spinner("🤖 Generating answer..."):
            result = qa.invoke({"query": query})
            st.subheader("Answer:")
            st.write(result["result"])



Overwriting app.py


In [46]:
!pip install pyngrok==4.1.1


In [ ]:
from pyngrok import ngrok, conf

conf.get_default().auth_token = "hf_XXXXXXXXXXXXXXXXXXXXXXXXXXXX"


In [71]:
# ① Start streamlit on port 80
get_ipython().system_raw('streamlit run app.py --server.port 80 &')


In [73]:
# ② Then create the ngrok tunnel
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(80, bind_tls=True).public_url
print("App available at:", public_url)

App available at: https://2598-35-230-54-57.ngrok-free.app
